In [1]:
# Imports
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from selenium import webdriver
from time import sleep
import random

In [102]:
# Declaring URL, WebDriver location, minimum request and sleep interval
url = "https://www.mycareersfuture.sg/search?search=Big%20Data&sortBy=new_posting_date&page={}"
chromedriver = "chromedriver.exe"
minreq = 1200
sleepy = random.randint(3, 7)

In [103]:
# Execution of URL in WebDriver and retrieve source
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url.format(0))
sleep(sleepy)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [104]:
# Dismiss Cookies Warning
button = driver.find_element_by_xpath('//*[@id="dismiss-button"]')
button.click()
sleep(random.randint(3, 7))

# Dismiss Skills to be Added Popup
button = driver.find_element_by_xpath('//*[@class="tr pointer OverlayNavigation__icon-cross___1wfSE"]')
button.click()
sleep(random.randint(3, 7))
soup = BeautifulSoup(driver.page_source, 'lxml')

In [105]:
# CareersFuture code begins here #
job_id = []
job_title = []
coy_name = []
coy_rating = []
review_count = []
job_location = []
alternate_source = []
min_salary = []
max_salary = []
salary_type = []
days_posted = []
job_desc = []
job_req = []
job_type = []
job_industry = []
job_url = []
job_count = 0
jip_count = 0
is_responsive = []
is_contract = []
is_internship = []
is_temporary = []
is_permanent = []

while job_count < minreq:
    list_of_jobs = soup.findAll("p", {"name":"company"})
    if jip_count < len(list_of_jobs):
        # ID
        job_id_var = soup.findAll("div", "absolute top-0 right-0 JobCard__bookmark-box___pdZMT")
        for x in range(len(job_id_var)):
            job_id.append(job_id_var[x].div.input["id"])
        # Job Title
        job_title_var = soup.findAll("h1", {"name":"job_title"})
        for x in range(len(job_title_var)):
            job_title.append(job_title_var[x].text)
        # Company Name
        company_name_var = soup.findAll("p", {"name":"company"})
        for x in range(len(company_name_var)):
            coy_name.append(company_name_var[x].text)
        # Reviews
        review_count.append(0)
        # Job Links
        job_link_var = soup.findAll("a", "bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3")
        for x in range(len(job_link_var)):
            job_url.append("https://www.mycareersfuture.sg"+job_link_var[x]["href"])
            # Next Level Stuff
            button = driver.find_element_by_xpath('//*[@href="'+job_link_var[x]["href"]+'"]')
            button.click()
            sleep(random.randint(3, 7))
            soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
            # Location
            job_location_var = soup_level2.findAll("p", {"id":"address"})
            if len(job_location_var) > 0:
                job_location.append(job_location_var[0].text)
            else:
                job_location.append(None)
            # Alternative Source (if any?)
            alternate_source_var = soup_level2.findAll("p", {"id": "seniority"})
            if len(alternate_source_var) > 0:
                alternate_source.append(alternate_source_var[0].text)
            else:
                alternate_source.append("Unknown")
            # Salary
            salary_var = soup_level2.findAll("span", "dib")
            if len(salary_var) > 0:
                if len(salary_var[1].text.split("to")) == 2:  # If there are 2 numbers given
                    minimum = int("".join(salary_var[1].text.split("to")[0].split("$")[1].split(",")))
                    maximum = int("".join(salary_var[1].text.split("to")[1].split("$")[1].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(maximum)
                    salary_type.append(salary_var[-2].text)
                elif salary_var[1].text.split("to")[0] == "Salary undisclosed": # Special Case (Non Recruiter)
                    min_salary.append(None)
                    max_salary.append(None)
                    salary_type.append(None)
                elif salary_var[2].text.split("to")[0] == "Salary undisclosed": # Special Case (Recruiter)
                    min_salary.append(None)
                    max_salary.append(None)
                    salary_type.append(None)
                else:  # If there is only 1 number given
                    minimum = int("".join(salary_var[2].text.split("to")[0].split("$")[1].split(",")))
                    maximum = int("".join(salary_var[2].text.split("to")[1].split("$")[1].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(maximum)
                    salary_type.append(salary_var[-2].text)
            else:  # If there are 0 number given
                min_salary.append(0)
                max_salary.append(0)
            # Job Type
            job_type_var = soup_level2.findAll('p', {"id":"employment_type"})
            list1 = ["Contract", "Internship", "Temporary", "Permanent"]
            list2 = [is_contract, is_internship, is_temporary, is_permanent]
            if len(job_type_var) > 0:
                job_type.append(job_type_var[0].text)
                for num, pos in enumerate(list1):
                    if pos in job_type_var[0].text.split(", "):
                        list2[num].append(1)
                    else:
                        list2[num].append(0)
            else:
                job_type.append("Unknown")
                for num, pos in enumerate(list1):
                    list2[num].append(0)
            # Job Industry
            job_industry_var = soup_level2.findAll("p", {"id": "job-categories"})
            if len(job_industry_var) > 0:
                job_industry.append(job_industry_var[0].text)
            else:
                job_industry.append(None)
            # Job Description
            job_description_var = soup_level2.findAll("div", {"id":"description-content"})
            if job_description_var:
                job_desc.append(job_description_var[0].text)
            else:
                job_desc.append("None")
            # Job Requirements
            job_requirements_var = soup_level2.findAll("div", {"id":"requirements-content"})
            if job_requirements_var:
                job_req.append(job_requirements_var[0].text)
            else:
                job_req.append("None")
            # days
            days_posted_var = soup_level2.findAll("span", {"id":"last_posted_date"})
            if len(days_posted_var) > 0:
                text = days_posted_var[0].text.split("Posted ")[1]
                days_posted.append(text)
            else:
                days_posted.append("None")
                
            driver.back()
            job_count+=1
            jip_count+=1
            print(job_count)
            if job_count == minreq:
                break
    else:
        if (soup.findAll("span", {"type":"action"})[-1].text == "❯"):
            button = driver.find_element_by_xpath('//*[@type="action"][last()]')
            button.click()
            sleep(sleepy)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            jip_count = 0
            try:
                # Dismiss Add Skills Popup
                button = driver.find_element_by_xpath('//*[@class="joyride-tooltip__close"]')
                button.click()
            except:
                continue
        else:
            driver.close()
            print("Done")
            break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [106]:
# Formatting the DataFrame before saving to CSV
df = pd.DataFrame([job_id, job_url, job_title, job_type, coy_name, job_location, alternate_source, job_industry, min_salary, max_salary, salary_type, days_posted, is_contract, is_internship, is_temporary, is_permanent, job_desc, job_req],
             index=["Job ID", "URL", "Job Title", "Job Type", "Company Name", "Job Location", "Job Level", "Job Industry", "Min Salary", "Max Salary", "Salary Paid", "Date Posted", "Contract", "Internship", "Temporary", "Permanent", "Job Description", "Job Requirements"]).T
df["Search"] = "Big Data"

In [107]:
df

,Job ID,URL,Job Title,Job Type,Company Name,Job Location,Job Level,Job Industry,Min Salary,Max Salary,Salary Paid,Date Posted,Contract,Internship,Temporary,Permanent,Job Description,Job Requirements,Search
0,1700c9362ec866f265284154fd7eb9c0,https://www.mycareersfuture.sg/job/research-fe...,"Research Fellow (1-year contract), Living Anal...",Unknown,SINGAPORE MANAGEMENT UNIVERSITY,None,Unknown,None,0,0,Monthly,None,0,0,0,0,None,None,Big Data
1,6adabd55637d9f81f3b1d13d3cc4e91c,https://www.mycareersfuture.sg/job/research-fe...,"Research Fellow (1-year contract), Living Anal...",Contract,SINGAPORE MANAGEMENT UNIVERSITY,None,Senior Management,Education and Training,4000,8000,Monthly,01 Nov 2018,1,0,0,0,COMPANY DESCRIPTION Singapore Management Uni...,PhD in Computer Science and related disciplin...,Big Data
2,79e47a007c3a23e24e1fc3b76368e468,https://www.mycareersfuture.sg/job/consumer-ma...,Consumer & Market Insights (CMI) Manager / Sen...,"Permanent, Full Time",6ESTATES PTE. LTD.,79 AYER RAJAH CRESCENT 139955,Senior Management,"Information Technology, Marketing / Public Rel...",6000,7000,Monthly,01 Nov 2018,0,0,0,1,The CMI role will have 2 areas of focus with h...,"Bachelor’s degree in Marketing, Market Resea...",Big Data
3,7d180e3342d7d698d82d85a9ab2df0b9,https://www.mycareersfuture.sg/job/consumer-ma...,Consumer & Market Insights (CMI) Executive,"Permanent, Full Time",6ESTATES PTE. LTD.,79 AYER RAJAH CRESCENT 139955,Professional,"Information Technology, Marketing / Public Rel...",3600,4000,Monthly,01 Nov 2018,0,0,0,1,The CMI role will have 2 areas of focus with h...,"Bachelor’s degree in Marketing, Market Resea...",Big Data
4,e3814a741a23ef0db60ddcaab0038dbf,https://www.mycareersfuture.sg/job/teradata-ha...,Teradata & Hadoop tester,Contract,ITCAN PTE. LIMITED,"PRUDENTIAL TOWER, 30 CECIL STREET 049712",Professional,Banking and Finance,4500,7000,None,01 Nov 2018,1,0,0,0,- Min 5+ years of experience in Manual Testing...,- Required minimum 5 years of work experience ...,Big Data
5,1e43b14761ccbf926b8993fcd766d3de,https://www.mycareersfuture.sg/job/field-partn...,Field & Partner Marketing Manager - Software ...,Unknown,Company Undisclosed,None,Unknown,None,0,0,Monthly,None,0,0,0,0,None,None,Big Data
6,2f5e15011244667d17bca1013e9e58c5,https://www.mycareersfuture.sg/job/infocomm-of...,Infocomm Officer / Senior Infocomm Officer [...,Full Time,Inland Revenue Authority of Singapore,"REVENUE HOUSE, 55 NEWTON ROAD 307987",Unknown,"Information Technology, Public / Civil Service",None,None,Monthly,01 Nov 2018,0,0,0,0,Be Transformational. Make your mark. Create an...,None,Big Data
7,b0a7ed9f6b62e3d210bcb5e3066ff88e,https://www.mycareersfuture.sg/job/gcg-regiona...,GCG - Regional Lending Pricing Manager - 18064152,Full Time,CITIBANK N.A.,None,Middle Management,Banking and Finance,10000,17500,Monthly,01 Nov 2018,0,0,0,0,Job Purpose: To build a consistent pricing fra...,Knowledge/Experience/Technical skills: Unders...,Big Data
8,e071636558aa896fa4008a42be015313,https://www.mycareersfuture.sg/job/ios-develop...,iOS Developer,Full Time,SMOVE SYSTEMS PTE. LTD.,75 AYER RAJAH CRESCENT 139953,Executive,Information Technology,5000,6600,Monthly,01 Nov 2018,0,0,0,0,Please note: Applications for this role shall ...,"The Challenge Acting as part of the iOS team, ...",Big Data
9,9ce6a945fa82f51e6aab13355fb5fcfa,https://www.mycareersfuture.sg/job/bi-consulta...,BI Consultant,Full Time,INTELLECT MINDS PTE. LTD.,"ANSON CENTRE, 51 ANSON ROAD 079904",Executive,Information Technology,6500,8000,Monthly,31 Oct 2018,0,0,0,0,Company Overview Intellect Minds is a Singapor...,Requirement and Qualification Bachelor’s Degr...,Big Data


In [108]:
len(df.drop_duplicates())

321

In [109]:
# Saving the DataFrame as CSV
df.to_csv("careersfuture_bigdata1")

------
Testing

In [80]:
Selector(text=html).xpath('//*').extract()

[]

In [90]:
driver.find_element_by_xpath('//*[@class="card-list"]//a[@href]')

<selenium.webdriver.remote.webelement.WebElement (session="cb8f741627213a5af494130857ce5648", element="0.6659543454550094-4")>

In [34]:
url = "https://www.mycareersfuture.sg/job/senior-executive-data-services-singapore-land-authority-1dc412ae4cb015131753c20a17a4ab84"
chromedriver = "chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url.format(0))
sleep(random.randint(10, 20))
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [35]:
soup.findAll("div", {"id":"description-content"})[0].text

'Senior Executive, Data Services\nJob Scope:\n\nProvide system support to internal users. Manage IT vendors to ensure proper and timely closure of reported issues\nSupport internal business units in their data sharing efforts initiatives. Preparation of data for sharing in internal/external data sharing platforms\nCoordinate with internal stakeholders to prepare and compile data for reporting of corporate performance indicators\nConduct audit on spatial data. Work closely with internal users to ensure reported data issues are rectified\nPerform data and geo-spatial analysis to fulfil business user requirements\n\n\xa0\nRequirements:\n\nDegree in Information Technology or Geographic Information Systems (GIS), with good understanding on the concepts/applications of GIS\nWorking experience in IT and/or GIS projects\nKnowledge and working experience in data analysis/audits\nPro-active, resourceful, tech savvy and willing to learn\nMeticulous and analytical mind, hands-on and performance dr

In [7]:
# Dismiss Cookies Warning
button = driver.find_element_by_xpath('//*[@id="dismiss-button"]')
button.click()
sleep(random.randint(3, 7))

# Dismiss Skills to be Added Popup
button = driver.find_element_by_xpath('//*[@class="tr pointer OverlayNavigation__icon-cross___1wfSE"]')
button.click()
sleep(random.randint(3, 7))
soup = BeautifulSoup(driver.page_source, 'lxml')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="tr pointer OverlayNavigation__icon-cross___1wfSE"]"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17134 x86_64)


In [38]:
driver.find_element_by_xpath('//*[@type="action"][last()]').click()

In [28]:
soup.findAll("span", {"type":"action"})[-1]

<span class="f5-5 pv2 ph3 mh1 pointer dib black-70 hover-bg-white" type="action">❯</span>

In [21]:
# Dismiss Add Skills Popup
button = driver.find_element_by_xpath('//*[@class="joyride-tooltip__close"]')
button.click()
sleep(random.randint(3, 7))

In [253]:
# Company Name
coy_name_var = soup.findAll("p", {"name":"company"})
for x in range(len(coy_name_var)):
    print(x, coy_name_var[x].text)

0 SANDBOX CONSULTING PTE. LTD.
1 SANDBOX CONSULTING PTE. LTD.
2 Company Undisclosed
3 Company Undisclosed
4 DBS BANK LTD.
5 DBS BANK LTD.
6 TELEPATHY LABS PTE. LTD.
7 GROUPM ASIA PACIFIC HOLDINGS PTE. LTD.
8 DELFERS BUSINESS SOLUTIONS PTE. LTD.
9 GECO ASIA PTE. LTD.
10 Singapore Land Authority
11 Company Undisclosed
12 ITCAN PTE. LIMITED
13 AYP STAFFING PTE. LTD.
14 CHEIL SINGAPORE PTE. LTD.
15 ALLEGIS GROUP SINGAPORE PRIVATE LIMITED
16 SIEMENS PTE. LTD.
17 ALEPH PTE. LTD.
18 ACCENTURE PTE LTD
19 Company Undisclosed


In [254]:
# Job Title
job_title_var = soup.findAll("h1", {"name":"job_title"})
for x in range(len(job_title_var)):
    print(x, job_title_var[x].text)

0 Data Analyst
1 Data Engineer
2 Systems and Data Analyst, Manager (Procurement) (Ref 22672)
3 Data Engineer
4 AVP  /  Senior Associate, Data Analyst, Analytic Center of Excellence, Transformation Grp (180003G5)
5 VP  /  AVP, Data Analyst, Middle Office Technology, Technology and Operations  (180003CW)
6 Data Analyst
7 Support Analyst, Media Trading Data Warehouse and Reporting Systems - Application Support Analyst
8 Data analyst
9 Data Solution Analyst in Singapore (90496)
10 Assistant Manager, GeoAnalytics
11 IT Business Analyst (Data Acquisition & Product Team)
12 Data Analyst
13 Data Security Analyst
14 Data Scientist
15 Data Analyst – 12 month contract – Singapore
16 Data Analyst (64803)
17 Data Scientist
18 Big Data Consultant
19 Data Analyst


In [154]:
# Job ID
job_id_var = soup.findAll("div", "absolute top-0 right-0 JobCard__bookmark-box___pdZMT")
for x in range(len(job_id_var)):
    print(x, job_id_var[x].div.input["id"])

0 5713cadaebb8b5aaf74163a35e16dd38
1 25654d51fa59534e3ac5d5e283830ea1
2 8258bc679291c6770a417b55ffab1329
3 e7c30aceb1ee80d8f68c4ee5f5de9e98
4 b914690e3a0a71debdbac82d7ff29f2b
5 b6ce5fea905fa3becc53a18ddbb472db
6 cabc219f3c40553b2534fa9a30542ca0
7 caf7f9cf6ac488cfa6d5027baedb43ba
8 6c7d936d80c316128ecaf03f3217c260
9 e77ec71735dee75e3fa3c0a36376d861
10 00349398f3f4f0b7507717aacd0e9057
11 9c61796335060413ccc0e87783062345
12 d3c21c288026478b7415d615319301a2
13 c14323f7551d6a40af3043bf45b63a49
14 7647b1184c87a6bf8bc5bb6fba2992f4
15 a1c1ecc881c636ca9aa0ee57ec150d0c
16 d97ad61da329ce17fa65bd93a76e73e5
17 fac9e1e7861434a33ac12bd4cddf8d36
18 4e248382f8a074f6ed3b6a52932a0952
19 17e5d1e82f31be1e5aa387e7a2d335e4


In [7]:
# Job Links
job_link_var = soup.findAll("a", "bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3")
for x in range(len(job_link_var)):
    #print(x, "https://www.mycareersfuture.sg"+job_link_var[x]["href"])
    print(x, job_link_var[x]["href"])

0 /job/data-analyst-cognizant-technology-solutions-asia-pacific-0aa2c9ab2235e4a4162802a77ab8783a
1 /job/security-analyst-techcom-solutions-consultancy-76fc74a944b39a566124745658c1220a
2 /job/data-analyst-sandbox-consulting-5713cadaebb8b5aaf74163a35e16dd38
3 /job/data-engineer-sandbox-consulting-25654d51fa59534e3ac5d5e283830ea1
4 /job/systems-data-analyst-manager-8258bc679291c6770a417b55ffab1329
5 /job/data-engineer-e7c30aceb1ee80d8f68c4ee5f5de9e98
6 /job/avp-senior-associate-data-analyst-analytic-center-excellence-transformation-grp-dbs-bank-b914690e3a0a71debdbac82d7ff29f2b
7 /job/vp-avp-data-analyst-middle-office-technology-technology-operations-dbs-bank-b6ce5fea905fa3becc53a18ddbb472db
8 /job/data-analyst-telepathy-labs-cabc219f3c40553b2534fa9a30542ca0
9 /job/support-analyst-media-trading-data-warehouse-reporting-systems-application-support-analyst-groupm-asia-pacific-holdings-caf7f9cf6ac488cfa6d5027baedb43ba
10 /job/data-analyst-delfers-business-solutions-6c7d936d80c316128ecaf03f321

In [83]:
button = driver.find_element_by_xpath('//*[@href="/job/data-analyst-cognizant-technology-solutions-asia-pacific-0aa2c9ab2235e4a4162802a77ab8783a"]')
button.click()
sleep(random.randint(3, 7))
soup_level2 = BeautifulSoup(driver.page_source, 'lxml')

In [249]:
# Salary
salary_var = soup_level2.findAll("span", "dib")[1].text.split("to")
print("Min","".join(salary_var[0].split("$")[1].split(",")))
print("Max","".join(salary_var[1].split("$")[1].split(",")))

# Location
job_location_var = soup_level2.findAll("p", {"id":"address"})[0].text
print(job_location_var)

# Job Type
job_type_var = soup_level2.findAll('p', {"id":"employment_type"})[0].text
print(job_type_var)

# Job Description
job_desc_var = soup_level2.findAll('div', 'jobDescription w-100 v-top relative')[0].text
print(job_desc_var)

# days
days_posted_var = soup_level2.findAll("span", {"id":"last_posted_date"})[0].text
print(days_posted_var.split("Posted ")[1])

# Alternative Source (if any?)
alternate_source_var = soup_level2.findAll("p", {"id": "seniority"})[0].text
print(alternate_source_var)

Min 5500
Max 6000
TRIVEX, 8 BURN ROAD 369977
Permanent, Contract, Full Time
Roles & Responsibilities Design, build, launch and maintain efficient and reliable large-scale batch and real-time data pipelines with data processing frameworks  Integrate and collate data silos in a manner which is both scalable and compliant  Collaborate with Project Manager, Frontend Developers, UX Designers and Data Analyst to build scalable data-driven products  Responsible for developing backend APIs & working on databases to support the applications  Working in an Agile Environment that practices Continuous Integration and Delivery  Working closely with fellow developers through pair programming and code review processRequirements Experience and passion for data engineering in a big data environment using Cloud platforms - AWS, GCP or Azure  Experience with building production-grade data pipelines, ETL/ELT data integration  Interested in being the bridge between engineering and analytics  Know

In [88]:
# Salary
salary_var = soup_level2.findAll("span", "dib")
print(salary_var[-2].text)

Monthly


In [48]:
salary_var = soup_level2.findAll("span", "dib")
if len(salary_var) > 0:
    if len(salary_var[1].text.split("to")) == 2:  # If there are 2 numbers given
        minimum = int("".join(salary_var[1].text.split("to")[0].split("$")[1].split(",")))
        maximum = int("".join(salary_var[1].text.split("to")[1].split("$")[1].split(",")))
        min_salary.append(minimum)
        max_salary.append(maximum)
    else:  # If there is only 1 number given
        minimum = int("".join(salary_var[1].text.split("to")[0].split("$")[1].split(",")))
        min_salary.append(minimum)
        max_salary.append(minimum)
else:  # If there are 0 number given
    min_salary.append(0)
    max_salary.append(0)

29 Oct 2018


In [250]:
driver.back()

In [49]:
driver.close()

In [50]:
driver.switch_to_window(driver.window_handles[0])

In [20]:
soup_level2.findAll("div", "jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs")

[<div class="jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs">Permanent</div>]

In [21]:
# job type
job_type_var = soup_level2.findAll("div", "jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs")
list1 = ["Contract", "Internship", "Temporary", "Permanent"]
list2 = [is_contract, is_internship, is_temporary, is_permanent]
if job_type_var:
    print(job_type_var[0].text)
    for num, pos in enumerate(list1):
        if pos in job_type_var[0].text.split(", "):
            print(1)
        else:
            print(0)
else:
    print("Unknown")
    for num, pos in enumerate(list1):
        print(0)

Permanent
0
0
0
1


In [ ]:
### Debug Section

In [ ]:
# Job Title
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    print(no, item.findAll("a", {"data-tn-element":"jobTitle"})[0]["title"])

In [ ]:
# Job Company
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    print(no, item.findAll("span", {"class":"company"})[0].text.split("\n        ")[1])

In [ ]:
# Job IDs
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    print(no, item["id"])

In [ ]:
# Number of reviews
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    if len(item.findAll("span", "slNoUnderline")) > 0:
        print(no, int("".join(item.findAll("span", "slNoUnderline")[0].text.split(" ")[0].split(","))))
    else:
        print(no, 0)

In [ ]:
# Location
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    print(no, item.findAll(["span", "div"], "location")[0].text)

In [ ]:
# Salary Info
for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    if len(item.findAll("span", "no-wrap")) > 0:
        minimum = int("".join(item.findAll(["span"], "no-wrap")[0].text.split("$")[1].split(" ")[0].split(",")))
        maximum = int("".join(item.findAll(["span"], "no-wrap")[0].text.split("$")[2].split(" ")[0].split(",")))
        print(no, minimum, maximum) # min
    else:
        print(no, 0, 0)

In [ ]:
#driver.implicitly_wait(2)
driver.switch_to_window(driver.window_handles[0])
#driver.close()

In [ ]:
days = []

for no, item in enumerate(soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])):
    button = driver.find_element_by_xpath('//*[@id="'+ item["id"] +'"]')
    button.click()
    driver.switch_to_window(driver.window_handles[1])
    soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
    # days
    text = soup_level2.findAll("div", "jobsearch-JobMetadataFooter")[0].text.split("- ")
    if "ago" in text[0]:
        days.append(text[0].split(" ")[0])
    else:
        days.append(text[1].split(" ")[0])
    #days.append(soup_level2.findAll("div", "jobsearch-JobMetadataFooter")[0].text.split("- ")[1].split(" ")[0])
    sleep(5)
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    print(no)
    sleep(2)

days
#button.get_attribute("outerHTML")
#print(driver.current_url)
#soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
#subsoup.findAll("div")


In [ ]:
button = driver.find_element_by_xpath('//*[@id="pj_b4fd692261af3cdc"]')
button.click()
driver.switch_to_window(driver.window_handles[1])
soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
print(soup_level2)
#days.append(soup_level2.findAll("div", "jobsearch-JobMetadataFooter")[0].text.split("- ")[1].split(" ")[0])
#sleep(5)
#driver.close()
#driver.switch_to_window(driver.window_handles[0])
print(no)
#sleep(2)